In [1]:
from tabulate import tabulate
from itertools import zip_longest
import numpy as np


class Table:
    __slots__ = ["data"] 

    def __init__(self, data=None, types=None):
        if data:
            self.data = np.array(list(zip_longest(*data.values())), dtype=types)
        else:
            self.data = np.array([])

    @property
    def data_types(self):
        return self.data.dtype.descr

    @property
    def length(self):
        return len(self.data)

    @property
    def width(self):
        return len(self.data_types)

    @property
    def names(self):
        return list(self.data.dtype.names)

    @property
    def rows_range(self):
        return range(self.length)

    def __getitem__(self, selector):
        if type(selector) is tuple:
            #TODO write code for handling slices, itnegers strings and more.
            name, index = selector
            return self.data[name][index]
        elif type(selector) is str:
            return self.data[selector]

    def __setitem__(self, selector, value):
        if type(selector) is tuple:
            #TODO write code for handling slices, itnegers strings and more.
            name, index = selector
            if name not in self.names:
                self.new_column(name)
            self.data[name][index] = value
        elif type(selector) is str:
            if selector not in self.names:
                self.new_column(selector)
            value = self.value_to_list(value)
            self.align_length(value)
            self.data[selector] = value

    def value_to_list(self, value):
        if type(value) is not list:
            value = [value]
        return value

    def align_length(self, value):
        length_difference = len(value) - self.length
        if length_difference > 0:
            for _ in range(1, length_difference + 1):
                self.data.resize(self.length + 1)
                self.data[-1] = tuple([None] * self.width)

    def new_column(self, name, data_type="O"):
        arrays_list = [list(self.data[name]) for name in self.names]
        arrays_list.append([None] * self.length)
        types = self.data_types + [(name, data_type)]
        
        self.data = np.array(list(zip_longest(*arrays_list)), dtype=types)
    
    def __str__(self):
        return tabulate(
            self.data, headers="keys", showindex="always", tablefmt="github", disable_numparse=True
        )


In [2]:
data = {
    "ColA": [1,2,3,4], "ColB": [1], "ColC": [4,2,7,1],
    "ColD": [8,3], "ColE": [1,4,3,1,1,1], "ColF": [3,2,4,2]
}
types = [
    ("ColA", "O"), ("ColB", "O"), ("ColC", "O"), ("ColD", "O"), ("ColE", "O"), ("ColF", "O")
]

In [3]:
table = Table(data, types)

In [10]:
print(table)

|    | ColA   | ColB   | ColC   | ColD   | ColE   | ColF   | ColG   | ColH   | ColI   |
|----|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| 0  | 1      | 1      | 4      | 8      | 1      | 3      |        | 2.0    | a      |
| 1  | 2      |        | 2      | 3      | 4      | 2      |        | 2.0    | b      |
| 2  | 3      |        | 7      |        | 3      | 4      |        | 3.5    | c      |
| 3  | 4      |        | 1      |        | 1      | 2      |        | 3.0    | d      |
| 4  |        |        |        |        | 1      |        | False  |        | e      |
| 5  |        |        |        |        | 1      |        |        |        | f      |
| 6  |        |        |        |        |        |        |        |        | g      |


In [5]:
table["ColG", 4] = False

In [7]:
for idx in table.rows_range:
    if table["ColA", idx] and table["ColF", idx]:
        table["ColH", idx] = (table["ColA", idx] + table["ColF", idx]) / 2

In [9]:
table["ColI"] = ["a", "b", "c", "d", "e", "f", "g"]